# Racquet Features Cleaning Notebook

This notebook documents the process of cleaning the raw tennis racquet listing data. Here is an overview of the changes made:

1. Get rid of all junior racquets from df

2. Drop all duplicated columns

3. Convert the following columns from object to float or int using regex or str logic:
    - Head Size
    - Length
    - Strung Weight
    - Balance
        - Create two columns: racquet_balance_in and racquet_balance_HH_HL
    - Stiffness
    - Beam width
    - String Pattern
        - Create two columns: racquet_mains and racquet_crosses
    - String Tension
        - Create two columns: racquet_tension_lower and racquet_tension_upper'


In all of these sections, I tried my best to pull out my df manipulation into custom functions. As I continue to learn about data science development, I am learning the importance of modular code, and systemitizing my work. The functions in this notebook have been pulled out and moved into the `preprocessing.py` modules in `src/data`.

## Table of Contents
1. [Imports and data loading](#imports-and-data-loading)

2. [Removing all junior racquets](#removing-all-junior-racquets)

3. [Drop all duplicated columns](#drop-all-duplicated-columns)

4. [Convert chosen spec columns from obj to float](#convert-chosen-spec-columns-from-object-to-float)

    a. [Extract racquet head size](#extract-racquet-head-size)

    b. [Extract racquet length](#extract-racquet-length)

    c. [Extract racquet strung weight](#extract-racquet-strung-weight-oz)

    d. [Extract racquet balance](#extract-racquet-balance-inches)

    e. [Extract racquet stiffness](#extract-racquet-stiffness)

    f. [Extract racquet average beam width](#extract-racquet-average-beam-width)

    g. [Extract racquet mains and crosses values](#extract-racquet-mains-and-crosses-values)
    
    h. [Extract racquet tension](#extract-racquet-tension)

5. [Final touch ups](#final-touch-ups)

## Imports and data loading

In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.pardir)))

In [2]:
import datashelf.core as ds
import pandas as pd
from tqdm import tqdm
import numpy as np
import re

In [3]:
ds.ls(to_display="coll-files")

+------------------------+------------------------------------------------------------------+---------------------+----------------------+---------+-----------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------+-----------+
| name                   | hash                                                             | date_created        | date_last_modified   | tag     |   version | message                                                                                                                                                                                 | file_path                                                                                                       | deleted   |
+========================+============================

In [4]:
scraped_racquet_data_raw = ds.load(
    collection_name="racquets", 
    hash_value="55dabe54d8b602a3c993460db0bf085737dc2c78a148e6d9fe5ea09f75b0e8ef"
    )

In [5]:
raw_df = scraped_racquet_data_raw.copy()

In [6]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 390 entries, 0 to 389
Data columns (total 37 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   racquet_img      390 non-null    object 
 1   racquet_name     390 non-null    object 
 2   racquet_rating   337 non-null    float64
 3   racquet_price    390 non-null    float64
 4   racquet_desc     390 non-null    object 
 5   Head Size        361 non-null    object 
 6   Length           361 non-null    object 
 7   Strung Weight    307 non-null    object 
 8   Balance          309 non-null    object 
 9   Swingweight      309 non-null    float64
 10  Stiffness        308 non-null    object 
 11  Beam Width       309 non-null    object 
 12  Composition      360 non-null    object 
 13  Power Level      309 non-null    object 
 14  Stroke Style     309 non-null    object 
 15  Swing Speed      309 non-null    object 
 16  Racquet Colors   351 non-null    object 
 17  Grip Type       

In [7]:
raw_df.describe()

,racquet_rating,racquet_price,Swingweight,Balance:,Swingweight:,Stiffness:,Beam Width:,Composition:,Power Level:,Stroke Style:,Swing Speed:,Racquet Colors:,Grip Type:,String Pattern:,String Tension:
count,337.000000,390.000000,309.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,4.725223,199.357769,316.919094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,0.467566,87.426284,11.176416,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000,12.990000,270.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,4.600000,129.000000,310.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,4.900000,199.475000,318.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,5.000000,269.000000,325.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,5.000000,579.000000,345.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
raw_df.shape

(390, 37)

In [9]:
raw_df.columns

Index(['racquet_img', 'racquet_name', 'racquet_rating', 'racquet_price',
       'racquet_desc', 'Head Size', 'Length', 'Strung Weight', 'Balance',
       'Swingweight', 'Stiffness', 'Beam Width', 'Composition', 'Power Level',
       'Stroke Style', 'Swing Speed', 'Racquet Colors', 'Grip Type',
       'String Pattern', 'String Tension', 'Balance:', 'Swingweight:',
       'Stiffness:', 'Beam Width:', 'Composition:', 'Power Level:',
       'Stroke Style:', 'Swing Speed:', 'Racquet Colors:', 'Grip Type:',
       'String Pattern:', 'String Tension:', 'Age', 'Weight', 'Height',
       'Other', 'Strung  Weight'],
      dtype='object')

In [10]:
raw_df

,racquet_img,racquet_name,racquet_rating,racquet_price,racquet_desc,Head Size,Length,Strung Weight,Balance,Swingweight,...,Swing Speed:,Racquet Colors:,Grip Type:,String Pattern:,String Tension:,Age,Weight,Height,Other,Strung Weight
0,https://img.tennis-warehouse.com/watermark/rs....,Babolat Pure Drive 2025,4.9,289.00,The Pure Drive is popular for a reason. Boast...,100 in² / 645.16 cm²,27in / 68.58cm,11.2oz / 318g,12.99in / 32.99cm / 4 pts HL,317.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://img.tennis-warehouse.com/watermark/rs....,Babolat Pure Drive 98 2025,4.5,299.00,Originally launched in 2019 under the VS moni...,98 in² / 632.26 cm²,27in / 68.58cm,11.4oz / 323g,13.18in / 33.48cm / 3 pts HL,326.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://img.tennis-warehouse.com/watermark/rs....,Babolat Pure Drive 98 2-Pack 2025,5.0,579.00,This product is for 2 Pure Drive 98 racquets....,98 in² / 632.26 cm²,27in / 68.58cm,11.4oz / 323g,13.18in / 33.48cm / 3 pts HL,323.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://img.tennis-warehouse.com/watermark/rs....,Babolat Pure Drive Plus 2025,5.0,289.00,Babolat adds another chapter to one of the ga...,100 in² / 645.16 cm²,27.5in / 69.85cm,11.2oz / 318g,13in / 33.02cm / 6 pts HL,325.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://img.tennis-warehouse.com/watermark/rs....,Babolat Pure Drive Team 2025,5.0,269.00,The Pure Drive Team 2025 is defined by its us...,100 in² / 645.16 cm²,27in / 68.58cm,10.6oz / 301g,12.85in / 32.64cm / 5 pts HL,308.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,https://img.tennis-warehouse.com/watermark/rs....,Solinco Blackout 285,4.5,229.99,Introducing the Blackout 285! Like the heavie...,100 in² / 645.16 cm²,27in / 68.58cm,10.6oz / 301g,13.38in / 33.99cm / 1 pts HL,315.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
386,https://img.tennis-warehouse.com/watermark/rs....,Solinco Blackout 300 XTD,4.8,229.99,"With the Blackout 300 XTD, Solinco takes the ...",100 in² / 645.16 cm²,27.5in / 69.85cm,11.3oz / 320g,12.8in / 32.51cm / 8 pts HL,328.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
387,https://img.tennis-warehouse.com/watermark/rs....,Solinco Blackout 300 XTD+,5.0,229.99,"With the Blackout 300 XTD+, Solinco gives adv...",100 in² / 645.16 cm²,28in / 71.12cm,11.3oz / 320g,12.8in / 32.51cm / 10 pts HL,333.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
388,https://img.tennis-warehouse.com/watermark/rs....,Lacoste L23,4.5,199.00,Introducing the Lascoste L23! Following on th...,100 in² / 645.16 cm²,27in / 68.58cm,11.1oz / 315g,12.9in / 32.77cm / 5 pts HL,318.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
raw_df["racquet_brand"] = raw_df["racquet_name"].apply(lambda x: x.split(" ")[0])
_new_col_order = ["racquet_brand"] + [col for col in raw_df.columns if col != "racquet_brand"]
raw_df = raw_df[_new_col_order]

raw_df["racquet_brand"].unique()

array(['Babolat', 'Wilson', 'Head', 'Yonex', 'Prince', 'Ektelon',
       'Tecnifibre', 'Dunlop', 'Volkl', 'ProKennex', 'Solinco', 'Lacoste'],
      dtype=object)

## Removing all junior racquets

In this section, I filter out all racquets that contain the word "Junior" in their name. These racquets seem to have NA values for most of their specs and are not relevant to my goal of creating a search product for adult tennis players.

In [12]:
junior_df = raw_df[raw_df["racquet_name"].str.contains("Junior")]

junior_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 69 entries, 37 to 318
Data columns (total 38 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   racquet_brand    69 non-null     object 
 1   racquet_img      69 non-null     object 
 2   racquet_name     69 non-null     object 
 3   racquet_rating   21 non-null     float64
 4   racquet_price    69 non-null     float64
 5   racquet_desc     69 non-null     object 
 6   Head Size        52 non-null     object 
 7   Length           52 non-null     object 
 8   Strung Weight    0 non-null      object 
 9   Balance          0 non-null      object 
 10  Swingweight      0 non-null      float64
 11  Stiffness        0 non-null      object 
 12  Beam Width       0 non-null      object 
 13  Composition      52 non-null     object 
 14  Power Level      0 non-null      object 
 15  Stroke Style     0 non-null      object 
 16  Swing Speed      0 non-null      object 
 17  Racquet Colors   43 n

In [13]:
no_junior_df = raw_df.merge(junior_df, how = "outer", indicator=True)
no_junior_df = no_junior_df[no_junior_df["_merge"]=="left_only"].drop(columns="_merge")

no_junior_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 321 entries, 4 to 389
Data columns (total 38 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   racquet_brand    321 non-null    object 
 1   racquet_img      321 non-null    object 
 2   racquet_name     321 non-null    object 
 3   racquet_rating   316 non-null    float64
 4   racquet_price    321 non-null    float64
 5   racquet_desc     321 non-null    object 
 6   Head Size        309 non-null    object 
 7   Length           309 non-null    object 
 8   Strung Weight    307 non-null    object 
 9   Balance          309 non-null    object 
 10  Swingweight      309 non-null    float64
 11  Stiffness        308 non-null    object 
 12  Beam Width       309 non-null    object 
 13  Composition      308 non-null    object 
 14  Power Level      309 non-null    object 
 15  Stroke Style     309 non-null    object 
 16  Swing Speed      309 non-null    object 
 17  Racquet Colors   308 

In [14]:
raw_df.shape[0] - junior_df.shape[0] == no_junior_df.shape[0]

True

In [15]:
no_junior_df[no_junior_df["Head Size"].isna()==True]

,racquet_brand,racquet_img,racquet_name,racquet_rating,racquet_price,racquet_desc,Head Size,Length,Strung Weight,Balance,...,Swing Speed:,Racquet Colors:,Grip Type:,String Pattern:,String Tension:,Age,Weight,Height,Other,Strung Weight
12,Babolat,https://img.tennis-warehouse.com/watermark/rs....,Babolat Boost Drive W,5.0,109.00,This racquet comes pre-strung for added conve...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,Babolat,https://img.tennis-warehouse.com/watermark/rs....,Babolat Boost Wimbledon,4.0,129.00,This racquet comes pre-strung for added conve...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68,Dunlop,https://img.tennis-warehouse.com/watermark/rs....,Dunlop SX 300 Lite,4.4,249.99,Updated with a new mold and enhanced dampenin...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,Dunlop,https://img.tennis-warehouse.com/watermark/rs....,Dunlop SX 300 LS,5.0,249.99,Dunlop gives intermediate players an easier p...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114,Head,https://img.tennis-warehouse.com/watermark/rs....,Head Extreme Elite 2024,4.5,174.00,"With the Extreme Elite 2024, Head puts the ea...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121,Head,https://img.tennis-warehouse.com/watermark/rs....,Head Radical MP 2021,4.9,164.00,Head adds another chapter to one of their mos...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
145,Head,https://img.tennis-warehouse.com/watermark/rs....,Head IG Boom XCeed,4.5,109.00,Introducing the Head IG Boom XCeed Racquet! W...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
174,Prince,https://img.tennis-warehouse.com/watermark/rs....,Prince Twistpower X100,4.6,179.00,"Introducing the Prince Twistpower X100, a lig...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
188,ProKennex,https://img.tennis-warehouse.com/watermark/rs....,ProKennex Black Ace 315,5.0,199.00,With roots that extend back 40 years to the o...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
339,Yonex,https://img.tennis-warehouse.com/watermark/rs....,Yonex EZONE 115 (2025),5.0,230.00,Introducing the EZONE 115 (2025)! New to the ...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
no_junior_df["String Tension"].isna().sum() / no_junior_df.shape[0]

0.04672897196261682

## Drop all duplicated columns

In this section, I find all duplicated columns by looking at percentage of NA values. In this case, I use 95% as the cuttoff threshold for dropping a column. This effectively eliminated all of the duplicates as well as the "Other" column which only had 1 or 2 non-null entries.

In [17]:
cols_to_drop = []

for col in no_junior_df.columns:
    if no_junior_df[col].isna().sum() == 0:
        pass
    elif no_junior_df[col].isna().sum() / no_junior_df.shape[0] > 0.95:
        cols_to_drop.append(col)
        print(f"{col} is {(no_junior_df[col].isna().sum()/no_junior_df.shape[0]).round(2)*100}% NAs.\
              \n Drop this column.")
    else:
        print(f"{col} is {(no_junior_df[col].isna().sum()/no_junior_df.shape[0]).round(2)*100}% NAs")

print(f"Drop the following columns: {', '.join(cols_to_drop)}")

racquet_rating is 2.0% NAs
Head Size is 4.0% NAs
Length is 4.0% NAs
Strung Weight is 4.0% NAs
Balance is 4.0% NAs
Swingweight is 4.0% NAs
Stiffness is 4.0% NAs
Beam Width is 4.0% NAs
Composition is 4.0% NAs
Power Level is 4.0% NAs
Stroke Style is 4.0% NAs
Swing Speed is 4.0% NAs
Racquet Colors is 4.0% NAs
Grip Type is 4.0% NAs
String Pattern is 4.0% NAs
String Tension is 5.0% NAs
Balance: is 100.0% NAs.              
 Drop this column.
Swingweight: is 100.0% NAs.              
 Drop this column.
Stiffness: is 100.0% NAs.              
 Drop this column.
Beam Width: is 100.0% NAs.              
 Drop this column.
Composition: is 100.0% NAs.              
 Drop this column.
Power Level: is 100.0% NAs.              
 Drop this column.
Stroke Style: is 100.0% NAs.              
 Drop this column.
Swing Speed: is 100.0% NAs.              
 Drop this column.
Racquet Colors: is 100.0% NAs.              
 Drop this column.
Grip Type: is 100.0% NAs.              
 Drop this column.
String Patte

In [18]:
na_dropped_df = no_junior_df.drop(columns = cols_to_drop)
na_dropped_df = na_dropped_df.reset_index().drop(columns = "index")
na_dropped_df

,racquet_brand,racquet_img,racquet_name,racquet_rating,racquet_price,racquet_desc,Head Size,Length,Strung Weight,Balance,...,Stiffness,Beam Width,Composition,Power Level,Stroke Style,Swing Speed,Racquet Colors,Grip Type,String Pattern,String Tension
0,Babolat,https://img.tennis-warehouse.com/watermark/rs....,Babolat Boost Aero Pink,5.0,119.0,This racquet comes pre-strung for added conve...,102 in² / 658.06 cm²,27in / 68.58cm,9.7oz / 275g,13.7in / 34.8cm / 2 pts HH,...,65,23mm / 26mm / 23mm,Graphite,Low-Medium,Medium-Full,Medium-Fast,Black/Pink,Babolat Uptake,16 Mains / 19 CrossesMains skip,50-55 pounds
1,Babolat,https://img.tennis-warehouse.com/watermark/rs....,Babolat Boost Aero,5.0,119.0,This racquet comes pre-strung for added conve...,102 in² / 658.06 cm²,27in / 68.58cm,9.7oz / 275g,13.7in / 34.8cm / 2 pts HH,...,65,23mm / 26mm / 23mm,Graphite,Low-Medium,Medium-Full,Medium-Fast,Black/Yellow,Babolat Uptake,16 Mains / 19 Crosses\n\n\nMains skip,50-55 pounds
2,Babolat,https://img.tennis-warehouse.com/watermark/rs....,Babolat Pure Aero 2023,4.6,289.0,"With its ""best of class"" combination of speed...",100 in² / 645.16 cm²,27in / 68.58cm,11.2oz / 318g,12.99in / 32.99cm / 4 pts HL,...,66,23mm / 26mm / 23mm,Graphite,Low-Medium,Medium-Full,Medium-Fast,Yellow/Black,Babolat Syntec Pro,16 Mains / 19 Crosses\n\n\nMains skip,50-59 pounds
3,Babolat,https://img.tennis-warehouse.com/watermark/rs....,Babolat Pure Aero 98 2023,4.9,299.0,Babolat adds another chapter to the most cont...,98 in² / 632.26 cm²,27in / 68.58cm,11.4oz / 323g,12.79in / 32.49cm / 6 pts HL,...,66,21mm / 23mm / 22mm,Graphite,Low-Medium,Medium-Full,Medium-Fast,Black/Yellow,Syntec Pro,16 Mains / 20 Crosses\n\n\nMains skip,50-59 pounds
4,Babolat,https://img.tennis-warehouse.com/watermark/rs....,Babolat Pure Aero Lite 2023,4.4,269.0,Updated with a softer feel and wider string s...,100 in² / 645.16 cm²,27in / 68.58cm,10oz / 283g,13.38in / 33.99cm / 1 pts HL,...,65,23mm / 26mm / 23mm,Graphite,Low-Medium,Medium-Full,Medium-Fast,"Yellow, Grey, White",Babolat Syntec Pro,16 Mains / 19 CrossesMains skip,50-59 pounds
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316,Yonex,https://img.tennis-warehouse.com/watermark/rs....,Yonex VCORE 98+,5.0,305.0,Yonex adds another chapter to the VCORE 98+! ...,98 in² / 632.26 cm²,27.5in / 69.85cm,11.4oz / 323g,13.18in / 33.48cm / 5 pts HL,...,62,23mm / 23mm / 21mm,2G-Namd FlexForce/H.M. Graphite,Low-Medium,Medium-Full,Medium-Fast,Red,Yonex Synthetic,16 Mains / 19 Crosses\n\n\nMains skip,45-60 pounds
317,Yonex,https://img.tennis-warehouse.com/watermark/rs....,Yonex VCORE 100L,4.2,305.0,"With the 2023 version of the VCORE 100L, Yone...",100 in² / 645.16 cm²,27in / 68.58cm,10.5oz / 298g,13.38in / 33.99cm / 1 pts HL,...,66,25.3mm / 25.3mm / 22mm,2G-Namd FlexForce/H.M. Graphite,Low-Medium,Medium-Full,Medium-Fast,Red,Yonex Synthetic,16 Mains / 19 CrossesMains skip,45-60 pounds
318,Yonex,https://img.tennis-warehouse.com/watermark/rs....,Yonex VCORE 95,4.8,305.0,Yonex adds another chapter to the VCORE 95! L...,95 in² / 612.9 cm²,27in / 68.58cm,11.5oz / 326g,12.59in / 31.98cm / 7 pts HL,...,61,22mm / 22mm / 21mm,2G-Namd FlexForce/H.M. Graphite,Low,Full,Fast,Red,Yonex Synthetic,16 Mains / 20 Crosses\n\n\nMains skip,45-60 pounds
319,Yonex,https://img.tennis-warehouse.com/watermark/rs....,Yonex VCORE 98,4.8,305.0,"With its redesigned beam and softer feel, the...",98 in² / 632.26 cm²,27in / 68.58cm,11.4oz / 323g,12.79in / 32.49cm / 6 pts HL,...,62,23mm / 23mm / 21mm,2G-Namd FlexForce/H.M. Graphite,Low-Medium,Medium-Full,Medium-Fast,Red,Yonex Synthetic,16 Mains / 19 CrossesMains skip,45-60 pounds


In [ ]:
# ds.save(
#     df = na_dropped_df,
#     collection_name="racquets",
#     name="no_nj_no_na_data",
#     tag="intermediate",
#     message = "Removed all junior racquets. Dropped all columns with more than 95% NA values."
# )
print("Already saved as CSV")

2025-07-28 19:55:21,862 - INFO - Save as CSV (1.07 MB)
2025-07-28 19:55:21,886 - INFO - no_nj_no_na_data added to racquets


0

# Convert chosen spec columns from object to float

In this section I convert the specified columns (below) from str to float (creating new columns in the process). I create various custom functions to systemize this process. Finally, I drop the str columns that were converted, and standardize the column names.

Recall that these are the columns I need to convert:

- Head Size
- Length
- Strung Weight
- Balance
    - Create two columns: racquet_balance_in and racquet_balance_HH_HL
- Stiffness
- Beam width
- String Pattern
    - Create two columns: racquet_mains and racquet_crosses
- String Tension
    - Create two columns: racquet_tension_lower and racquet_tension_upper

One similarity I noticed was that "Head Size", "Length", and "Balance" all have a measurement in inches and centimeters separated by a /.

Since I want to make two columns from Balance, one with its balance in inches and another with a dynamic measure of head heaviness, I can use a function that extracts the first term followed by "in" or "in^2" for three columns (Head Size, Length, and one of the two Balance columns I want to make).

In [19]:
regex_df = na_dropped_df.copy()
str_cols = [] 
for col in regex_df.columns:
    if regex_df[col].dtype == "object" and "racquet_" not in col:
        str_cols.append(col)
    else:
        pass
str_cols

#Keep Composition, Power Level, Stroke Style, Swing Speed, Racquet Colors, Grip Type

#Drop Head Size, Length, Strung Weight, Balance, Beam Width, String Pattern, String Tension

['Head Size',
 'Length',
 'Strung Weight',
 'Balance',
 'Stiffness',
 'Beam Width',
 'Composition',
 'Power Level',
 'Stroke Style',
 'Swing Speed',
 'Racquet Colors',
 'Grip Type',
 'String Pattern',
 'String Tension']

### Extract racquet head size

In [20]:
regex_df["racquet_head_size_sq_in"] = (
    regex_df["Head Size"]\
        .str.extract(r"(\d+\.?\d*)\s*(?:in²|in|sq\s*in)")\
            .astype(float)
)

In [21]:
regex_df[["Head Size", "racquet_head_size_sq_in"]]

,Head Size,racquet_head_size_sq_in
0,102 in² / 658.06 cm²,102.0
1,102 in² / 658.06 cm²,102.0
2,100 in² / 645.16 cm²,100.0
3,98 in² / 632.26 cm²,98.0
4,100 in² / 645.16 cm²,100.0
...,...,...
316,98 in² / 632.26 cm²,98.0
317,100 in² / 645.16 cm²,100.0
318,95 in² / 612.9 cm²,95.0
319,98 in² / 632.26 cm²,98.0


### Extract racquet length

In [22]:
regex_df["racquet_length_in"] = (
    regex_df["Length"]\
    .str.extract(r"(\d+\.?\d*)\s*(?:in²|in|sq\s*in)")\
    .astype(float)
)

regex_df[["Length", "racquet_length_in"]]

,Length,racquet_length_in
0,27in / 68.58cm,27.0
1,27in / 68.58cm,27.0
2,27in / 68.58cm,27.0
3,27in / 68.58cm,27.0
4,27in / 68.58cm,27.0
...,...,...
316,27.5in / 69.85cm,27.5
317,27in / 68.58cm,27.0
318,27in / 68.58cm,27.0
319,27in / 68.58cm,27.0


### Extract racquet strung weight (oz)

In [23]:
regex_df["racquet_strung_weight_oz"] = (
    regex_df["Strung Weight"]\
    .str.extract(r"(\d+\.?\d*)\s*")\
    .astype(float)
)

regex_df[["Strung Weight", "racquet_strung_weight_oz"]]

,Strung Weight,racquet_strung_weight_oz
0,9.7oz / 275g,9.7
1,9.7oz / 275g,9.7
2,11.2oz / 318g,11.2
3,11.4oz / 323g,11.4
4,10oz / 283g,10.0
...,...,...
316,11.4oz / 323g,11.4
317,10.5oz / 298g,10.5
318,11.5oz / 326g,11.5
319,11.4oz / 323g,11.4


### Extract racquet balance (inches)

In [24]:
# Extract inches Balance value
regex_df["racquet_balance_in"] = (
    regex_df["Balance"]\
    .str.extract(r"(\d+(?:\.\d+)?)\s*in\b").
    astype(float)
)

# Extract Balance number and label separately
extracted = regex_df["Balance"].str.extract(
    r"(\d+(?:\.\d+)?)\s*(?:pts\s*)?(HL|HH|EB)\b"
)

extracted.columns = ["value", "label"]

extracted["value"] = extracted["value"].astype(float)

def apply_balance_sign(row):
    if row["label"] == "HL":
        return row['value']
    elif row["label"] == "HH":
        return -row["value"]
    elif row["label"] == "EB":
        return 0.0
    return None

regex_df["racquet_balance_HH_HL"] = extracted.apply(apply_balance_sign, axis = 1)

regex_df[["Balance", "racquet_balance_in", "racquet_balance_HH_HL"]]

,Balance,racquet_balance_in,racquet_balance_HH_HL
0,13.7in / 34.8cm / 2 pts HH,13.70,-2.0
1,13.7in / 34.8cm / 2 pts HH,13.70,-2.0
2,12.99in / 32.99cm / 4 pts HL,12.99,4.0
3,12.79in / 32.49cm / 6 pts HL,12.79,6.0
4,13.38in / 33.99cm / 1 pts HL,13.38,1.0
...,...,...,...
316,13.18in / 33.48cm / 5 pts HL,13.18,5.0
317,13.38in / 33.99cm / 1 pts HL,13.38,1.0
318,12.59in / 31.98cm / 7 pts HL,12.59,7.0
319,12.79in / 32.49cm / 6 pts HL,12.79,6.0


### Extract racquet stiffness

In [25]:
regex_df["racquet_stiffness"] = regex_df["Stiffness"]
regex_df['racquet_stiffness'] = regex_df['racquet_stiffness'].replace('N/A (very low)', np.nan)

regex_df["racquet_stiffness"] = regex_df["racquet_stiffness"].astype(float)

regex_df[["Stiffness", "racquet_stiffness"]]

,Stiffness,racquet_stiffness
0,65,65.0
1,65,65.0
2,66,66.0
3,66,66.0
4,65,65.0
...,...,...
316,62,62.0
317,66,66.0
318,61,61.0
319,62,62.0


### Extract racquet average beam width

In [26]:
# Get average beam width as proxy for 3-value beam width field

def average_beam_width(value):
    if isinstance(value, str):
        parts = value.split("/")
        numbers = []
        for part in parts:
            cleaned = part.strip().replace("mm", "")
            if cleaned:
                try:
                    numbers.append(float(cleaned))
                except ValueError:
                    pass
        if numbers:
            return sum(numbers) / len(numbers)
        else:
            return float("nan")
    else:
        return float("nan")
    
regex_df["racquet_avg_beam_width"] = regex_df["Beam Width"].apply(average_beam_width)

regex_df[["Beam Width", "racquet_avg_beam_width"]]

,Beam Width,racquet_avg_beam_width
0,23mm / 26mm / 23mm,24.000000
1,23mm / 26mm / 23mm,24.000000
2,23mm / 26mm / 23mm,24.000000
3,21mm / 23mm / 22mm,22.000000
4,23mm / 26mm / 23mm,24.000000
...,...,...
316,23mm / 23mm / 21mm,22.333333
317,25.3mm / 25.3mm / 22mm,24.200000
318,22mm / 22mm / 21mm,21.666667
319,23mm / 23mm / 21mm,22.333333


### Extract racquet mains and crosses values

In [27]:
# Extract main and cross values, assign to relevant column in series, and pass series to df

def extract_mains_crosses(value):
    mains = np.nan
    crosses = np.nan
    
    if isinstance(value, str) and value.strip():
        
        mains_regex = re.search(r'(\d+)\s*Mains', value, re.IGNORECASE)
        crosses_regex = re.search(r'(\d+)\s*Crosses', value, re.IGNORECASE)
        
        if mains_regex:
            mains = float(mains_regex.group(1))
            
        if crosses_regex:
            crosses = float(crosses_regex.group(1))
            
    return pd.Series([mains, crosses])


regex_df[["racquet_mains", "racquet_crosses"]] = regex_df["String Pattern"].apply(extract_mains_crosses)

regex_df[["String Pattern", "racquet_mains", "racquet_crosses"]]
        

,String Pattern,racquet_mains,racquet_crosses
0,16 Mains / 19 CrossesMains skip,16.0,19.0
1,16 Mains / 19 Crosses\n\n\nMains skip,16.0,19.0
2,16 Mains / 19 Crosses\n\n\nMains skip,16.0,19.0
3,16 Mains / 20 Crosses\n\n\nMains skip,16.0,20.0
4,16 Mains / 19 CrossesMains skip,16.0,19.0
...,...,...,...
316,16 Mains / 19 Crosses\n\n\nMains skip,16.0,19.0
317,16 Mains / 19 CrossesMains skip,16.0,19.0
318,16 Mains / 20 Crosses\n\n\nMains skip,16.0,20.0
319,16 Mains / 19 CrossesMains skip,16.0,19.0


### Extract racquet tension

In [28]:
def extract_tension_bounds(value):
    lower = np.nan
    upper = np.nan
    
    if isinstance(value, str) and value.strip():
        tension_regex = re.search(r'(\d+)\s*-\s*(\d+)', value)
        if tension_regex:
            lower = float(tension_regex.group(1))
            upper = float(tension_regex.group(2))
        
    return pd.Series([lower, upper])

regex_df[["racquet_tension_lower", "racquet_tension_upper"]] = regex_df["String Tension"].apply(extract_tension_bounds)

regex_df[["String Tension", "racquet_tension_lower", "racquet_tension_upper"]]



,String Tension,racquet_tension_lower,racquet_tension_upper
0,50-55 pounds,50.0,55.0
1,50-55 pounds,50.0,55.0
2,50-59 pounds,50.0,59.0
3,50-59 pounds,50.0,59.0
4,50-59 pounds,50.0,59.0
...,...,...,...
316,45-60 pounds,45.0,60.0
317,45-60 pounds,45.0,60.0
318,45-60 pounds,45.0,60.0
319,45-60 pounds,45.0,60.0


## Final touch ups

In [29]:
intermediate_df = regex_df.copy()
intermediate_df.drop(columns = ["Head Size", "Length", "Strung Weight", "Balance", "Beam Width", "String Pattern", "String Tension", "Stiffness"], inplace = True)
intermediate_df.rename(columns = {"Swingweight":"racquet_swingweight",
                                  "Composition":"racquet_composition",
                                  "Power Level":"racquet_power",
                                  "Stroke Style":"racquet_stroke_style",
                                  "Swing Speed":"racquet_swing_speed",
                                  "Racquet Colors":"racquet_colors",
                                  "Grip Type":"racquet_grip"},
                       inplace = True)

In [30]:
intermediate_df.columns

Index(['racquet_brand', 'racquet_img', 'racquet_name', 'racquet_rating',
       'racquet_price', 'racquet_desc', 'racquet_swingweight',
       'racquet_composition', 'racquet_power', 'racquet_stroke_style',
       'racquet_swing_speed', 'racquet_colors', 'racquet_grip',
       'racquet_head_size_sq_in', 'racquet_length_in',
       'racquet_strung_weight_oz', 'racquet_balance_in',
       'racquet_balance_HH_HL', 'racquet_stiffness', 'racquet_avg_beam_width',
       'racquet_mains', 'racquet_crosses', 'racquet_tension_lower',
       'racquet_tension_upper'],
      dtype='object')

In [31]:
from datashelf.core.config import get_allowed_tags
get_allowed_tags()

['raw', 'intermediate', 'cleaned', 'ad-hoc', 'final']

In [32]:
## Save to datashelf
# ds.save(
#     df = intermediate_df,
#     collection_name = "racquets",
#     name = "Basic Cleaned Data",
#     tag = "cleaned",
#     message = "Removed all junior racquets. Removed duplicate columns. Used regex to extract values for specially formatted columns. Standardized column naming. Dropped all non-preprocessed columns."
# )

print("Already saved as CSV")

Already saved as CSV


In [35]:
ds.ls("coll-files")

+------------------------+------------------------------------------------------------------+---------------------+----------------------+--------------+-----------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------+-----------+
| name                   | hash                                                             | date_created        | date_last_modified   | tag          |   version | message                                                                                                                                                                                 | file_path                                                                                                          | deleted   |
+========================+============